# Prashant Dutt

## Research question/interests
How does seasonality effect the market? <br>
Which sectors/ stocks/ ETFS are most sensitive and least sensitve to seasonality?

In [12]:
import pandas as pd
data = pd.read_csv("../data/raw/archive/ETFs/retl.us.txt", sep=",", header=0, names=["Date", "Open", "High", "Low", "Close", "Volume", "OpenInt"])
data.head()

,Date,Open,High,Low,Close,Volume,OpenInt
0,2010-07-14,3.0042,3.0842,3.0042,3.0283,923050,0
1,2010-07-15,3.0314,3.0788,3.0251,3.0717,1225763,0
2,2010-07-16,2.9498,2.9520,2.8828,2.8937,773018,0
3,2010-07-19,2.8696,2.8696,2.8588,2.8634,147921,0
4,2010-07-20,2.8409,2.9708,2.8409,2.9708,54019,0
